In [4]:
from dotenv import load_dotenv
from metagame import TradingClient
import os

load_dotenv()

client = TradingClient(
    api_url=os.environ["API_URL"],
    jwt=os.environ["JWT"],
    act_as=int(os.environ["ACT_AS"]),
)

In [5]:
import pandas as pd

def get_trades(state, market_id:int):
    if not state.markets[market_id].hasFullTradeHistory:
        client.get_full_trade_history(market_id)
        state = client.state()
    return state.markets[market_id].trades

def get_market_trades(market_name: str):
    state = client.state()
    market_id = state.market_name_to_id.get(market_name)
    if market_id is None:
        return pd.DataFrame()
    
    trades = get_trades(state, market_id)
    
    if not trades:
        return pd.DataFrame()
    df = pd.DataFrame(trades).set_index("id")
    df["buyer"] = df["buyer_id"].map(lambda x: state.accounts[x].name)
    df["seller"] = df["seller_id"].map(lambda x: state.accounts[x].name)
    df["timestamp"] = df["transaction_id"].map(lambda x: state.transactions[x])
    return df.drop(columns=["market_id", "buyer_id", "seller_id", "transaction_id"])


In [6]:
def get_market_positions(market_name: str):
    df = get_market_trades(market_name)
    if df.empty:
        return pd.DataFrame()
    return pd.concat([
        df.groupby("buyer")["size"].sum(),
        -df.groupby("seller")["size"].sum()
    ]).groupby(level=0).sum()

def get_multiple_market_positions(market_names: list[str]):
    return pd.DataFrame({market: get_market_positions(market) for market in market_names}).fillna(0)

In [7]:
import ipywidgets as widgets

def get_dice(refresh):
    markets = ["min_dice_c", "max_dice_c", "avg_dice_c"]
    return get_multiple_market_positions(markets)

widgets.interact(get_dice, refresh=widgets.Checkbox(description="Refresh Data"))

interactive(children=(Checkbox(value=False, description='Refresh Data'), Output()), _dom_classes=('widget-inte…

<function __main__.get_dice(refresh)>

In [8]:
market_name = "min_dice_c"
state = client.state()

market_id = state.market_name_to_id.get(market_name)

trades = get_trades(state, market_id)

In [12]:
for ii, market in state.markets.items():
    for trade in market.trades:
        print(trade)
        break

Trade(id=316, market_id=19, transaction_id=1833, price=4.0, size=1.0, buyer_id=66, seller_id=0)
Trade(id=318, market_id=20, transaction_id=1858, price=17.0, size=10.0, buyer_id=0, seller_id=0)
Trade(id=315, market_id=21, transaction_id=1819, price=9.0, size=1.0, buyer_id=0, seller_id=0)


In [ ]:
settle_price = state.markets[market_id].definition.closed.settle_price
print(f"Settle Price: {settle_price}")

: 

: 

In [ ]:
my_name = "dice_c_1"
my_id = next((ac for ac in state.accounts if ac.name == my_name)).id
my_id

: 

: 

In [ ]:
for tr in trades:
    if tr.seller_id == my_id:
        print('s',tr.price, tr.size, ( tr.price - settle_price ) * tr.size)
    elif tr.buyer_id == my_id:
        print('b', tr.price, tr.size, round((settle_price- tr.price ) * tr.size, 2))

: 

: 

In [ ]:
market_name = "min_dice_c"
df = get_market_trades(market_name)

: 

: 

In [ ]:
my_account = "dice_c_1"
df[df["buyer"] == my_account]

: 

: 